In [8]:
import csv
import json
import re
import ast

import pandas as pd
import requests

In [9]:
def getItem(category):
    if category is None:
        url = 'https://api.nookplaza.net/items'
    else:
        url = 'https://api.nookplaza.net/items?category={}'.format(category)
    r = requests.get(url)
    text = r.json()
    print('get {} {}'.format(category, len(text['results'])))
    return text['results']

In [10]:
def saveFile(category, output, dictName):
    with open('../../nook_link/src/data/{}.json'.format(category), 'wt') as fout:
        fout.write(json.dumps(output))

    with open('{}.json'.format(category), 'wt') as fout:
        fout.write(json.dumps(output))

    with open('{}.csv'.format(category), 'wt', encoding='utf-8', newline='') as fout:
        csvout = csv.writer(fout)

        output_csv = [dictName]
        for row in output:
            row_ = [row[name] for name in dictName]
            output_csv.append(row_)
        csvout.writerows(output_csv)
    
    print('save {} over'.format(category))

In [11]:
def getEn2twFn():
    nookplazaUrl = 'https://nookplaza.net'
    r = requests.get(nookplazaUrl)

    m = re.search('main\.dart\.js\?v=(\d*)', r.text)
    dartJs = m.group()

    mainJsUrl = '{}/{}'.format(nookplazaUrl, dartJs)
    r = requests.get(mainJsUrl)
    mainJs = r.text

    # 'C.Iz=new H.b(13,{},C.a,[P.a,P.a])'
    m = re.findall('new H\.b\(13,(.*),C\.a,\[P\.a,P\.a\]\)', mainJs)

    m = [item.encode("utf-8").decode('unicode-escape') for item in m]
    m = [ast.literal_eval(item) for item in m if not 'null' in item]

    nameMap = {}
    for item in m:
        nameMap[item["en-us"]] = item

    print('get {} i18n data'.format(len(nameMap.keys())))

    try:
        df = pd.read_excel('../tempData/Furniture.xlsx', sheetname=4)
    except:
        df = pd.read_excel('../tempData/Furniture.xlsx', sheet_name=4)

    EnTwTable = ['類別', '取得方式', '標籤', '主題', '款式', '樣式', '款式說明', '樣式說明', 'DIY相關', '藝術品', '作者/年代/材質', '藝術品真實圖', '村民']

    nameMapCustom = {}
    for EnTw in EnTwTable:
        itemType_en = df[EnTw].tolist()
        itemType_tw = df['{}_tw'.format(EnTw)].tolist()
        itemType_en = [x for x in itemType_en if str(x) != 'nan']
        itemType_tw = [x for x in itemType_tw if str(x) != 'nan']

        for item in zip(itemType_en, itemType_tw):
            if item[0] in nameMapCustom and item[1] != nameMapCustom[item[0]]['zh-tw']:
                print(item)
                print(nameMapCustom[item[0]])
                print('------\n')

            nameMapCustom[item[0]] = {
                'zh-tw': item[1]
            }

    print('Custom {} i18n data'.format(len(nameMapCustom.keys())))

    def en2twFn(en, Custom = False):
        if Custom == 'onlySite':
            return nameMap[en]

        if Custom:
            if en in nameMapCustom:
                return nameMapCustom[en]

            elif en.lower() in nameMapCustom:
                return nameMapCustom[en.lower()]

            elif (en[0].lower() + en[1:]) in nameMapCustom:
                return nameMapCustom[(en[0].lower() + en[1:])]

        if en.lower() in nameMap:
            return nameMap[en.lower()]

        elif (en[0].lower() + en[1:]) in nameMap:
            return nameMap[(en[0].lower() + en[1:])]

        elif en in nameMap:
            return nameMap[en]

        elif en in nameMapCustom:
                return nameMapCustom[en]

        elif en.lower() in nameMapCustom:
            return nameMapCustom[en.lower()]

        elif (en[0].lower() + en[1:]) in nameMapCustom:
            return nameMapCustom[(en[0].lower() + en[1:])]

        elif en.replace('The', 'the') in nameMap:
            return nameMap[en.replace('The', 'the')]

        elif en.replace('K.K. Tude', 'K.K. Étude') in nameMap:
            return nameMap[en.replace('K.K. Tude', 'K.K. Étude')]

        elif en.replace('Caf K.K.', 'Café K.K.') in nameMap:
            return nameMap[en.replace('Caf K.K.', 'Café K.K.')]

        print(en)
        raise en
        
    return en2twFn

In [20]:
en2twFn = getEn2twFn()

idMap = {}
name2fileNameMap = {}
allItem = getItem(None)
for item in allItem: 
    if item['name'].startswith("Present ("):
        continue
        
    if 'internalID' in item['content']:
        try:
            name = en2twFn(item['name'].replace(' (fake)', ''))['zh-tw']
            idMap[item['content']['internalID']] = {'name': name, 'filename': item['content']['image']}
            name2fileNameMap[item['name']] = item['content']['filename']
        except:
            pass
#             print(item['name'])

print('get {} id to Name data'.format(len(idMap.keys())))

get 5713 i18n data
Custom 1080 i18n data
get None 5746
Stache & glasses
Pacifier
Spoiled turnips
Bell bag
Turnips
Coin
get 4501 id to Name data


In [21]:
en2twFn('Dress-Up')

# nameMap['Tank']
# name2fileNameMap['Stormy-night wall']

{'zh-tw': '套裝'}

In [13]:
output = []
furnitures = getItem('Furniture')
recipes = getItem('Recipes')

DIYMap = {}
for diy in recipes:
    DIYMap[diy['name']] = diy['content']

for item in furnitures:
    if item["category"] == "Art":
        continue

    pickData = {'type': 'furnitures'}
    name = en2twFn(item['name'])
    
    pickData['id'] = int(item['content']['internalID'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']

    pickData['buy'] = item['content']['buy'] if 'buy' in item['content'] else None
    pickData['sell'] = item['content']['sell']

    pickData['category'] = en2twFn(item['content']['category'], True)['zh-tw']
    pickData['tag'] = en2twFn(item['content']['tag'], True)['zh-tw']
    pickData['themes'] = [en2twFn(theme, True)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
    
    pickData['interact'] = item['content']['interact']
    pickData['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
    pickData['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickData['DIY'] = item['content']['dIY']
#     pickData['diyInfo'] = {}
    pickData['diyInfoMaterials'] = []
    pickData['diyInfoObtainedFrom'] = []
    pickData['diyInfoSourceNotes'] = ''

    if item['content']['dIY']:
        diyInfo = DIYMap[item['name']]
        pickData['diyInfoMaterials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in diyInfo['materials']]
        pickData['diyInfoObtainedFrom'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in diyInfo['obtainedFrom']]
        
        try:
            pickData['diyInfoSourceNotes'] = '' if diyInfo['sourceNotes'] == '' else en2twFn(diyInfo['sourceNotes'], True)['zh-tw']
        except:
            pickData['diyInfoSourceNotes'] = diyInfo['sourceNotes']
        
    pickData['filename'] = item['content']['filename']
    
    if 'bodyTitle' in item['content'] and item['content']['bodyTitle']:
        pickData['bodyTitle'] = en2twFn(item['content']['bodyTitle'], True)['zh-tw']
    else:
        pickData['bodyTitle'] = '無'
        
    if 'bodyCustomize' in item['content']:
        pickData['bodyCustomize'] = item['content']['bodyCustomize']
    else:
        pickData['bodyCustomize'] = False
    
    if 'patternTitle' in item['content'] and item['content']['patternTitle']:
        pickData['patternTitle'] = en2twFn(item['content']['patternTitle'], True)['zh-tw']
    else:
        pickData['patternTitle'] = '無'
        
    if 'patternCustomize' in item['content']:
        pickData['patternCustomize'] = item['content']['patternCustomize']
    else:
        pickData['patternCustomize'] = False

    variations = {
        "bodys": {},
        "pattrens": {}
    }
    for variation in item['variations']:
        content = variation['content']
        bodyIndex, pattrenIndex = content['filename'].split('_')[-2:]
        
        bodyColor = en2twFn(content['bodyColor'], True)['zh-tw'] if 'bodyColor' in content and content['bodyColor'] is not None else ''
        patternColor = en2twFn(content['patternColor'], True)['zh-tw'] if 'patternColor' in content and content['patternColor'] is not None else ''
        
        variations['bodys'][bodyColor] = bodyIndex
        variations['pattrens'][patternColor] = pattrenIndex
        
    pickData['variations'] = variations
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'buy', 'sell', 'category', 'tag', 'themes', 'interact', 'size', 'obtainedFrom', 'DIY', 'diyInfoMaterials', 'diyInfoObtainedFrom', 'diyInfoSourceNotes']
dictName += ['filename', 'bodyTitle', 'patternTitle', 'bodyCustomize', 'patternCustomize', 'variations']

saveFile('furniture', output, dictName)

get Furniture 1114
get Recipes 595
Obtained during the main storyline during the villager house development quest if the native fruit is oranges
50% chance of being obtained during the main storyline during the villager house development quest
50% chance of being obtained during the main storyline during the villager house development quest
Obtained during the main storyline during the villager house development quest if the native fruit is peaches
Purchase Test Your DIY Skills from Nook's Cranny
Obtained during the main storyline during the villager house development quest if the native fruit is pears
50% chance of being obtained during the main storyline during the villager house development quest
Purchase Test Your DIY Skills from Nook's Cranny
Obtained during the main storyline during the villager house development quest if the native fruit is pears
Purchase Test Your DIY Skills from Nook's Cranny; Obtained during the main storyline during the villager house development quest
50% c

In [14]:
furnitures = getItem('Furniture')

ArtMap = {}
for item in furnitures:
    if item["category"] != "Art":
        continue
        
    pickArt = {'type': 'arts'}
    name = en2twFn(item['name'].replace(' (fake)', ''))

    pickArt['id'] = int(item['content']['internalID'])
    pickArt['category'] = en2twFn(item['category'], True)['zh-tw']
    pickArt['name_c'] = name['zh-tw']
    pickArt['name_j'] = name['ja-jp']
    pickArt['name_e'] = name['en-us']
    
    pickArt['artist'] = en2twFn(item['content']['artist'])['zh-tw']
    pickArt['filename'] = item['content']['filename']
    pickArt['interact'] = item['content']['interact']
    pickArt['itemCategory'] = item['content']['itemCategory']
    pickArt['museumDescription'] = item['content']['museumDescription']
    pickArt['obtainedFrom'] = '、'.join([en2twFn(item)['zh-tw'] for item in item['content']['obtainedFrom']])
    pickArt['realArtworkTitle'] = item['content']['realArtworkTitle']
    pickArt['realArtworkTitle_tw'] = en2twFn(item['content']['realArtworkTitle'])['zh-tw']
    pickArt['tag'] = en2twFn(item['content']['tag'], True)['zh-tw']
    pickArt['themes'] = [en2twFn(theme, True)['zh-tw'] for theme in item['content']['themes'] if theme != 'None']
    pickArt['size'] = '{}x{}'.format(item['content']['size']['rows'], item['content']['size']['cols'])
    pickArt['realArtImage'] = en2twFn(item['content']['filename'].replace('Fake', ''))['zh-tw']
    
    if not name['en-us'] in ArtMap:
        ArtMap[name['en-us']] = {}
        
    ArtMap[name['en-us']][str(item['content']['genuine'])] = pickArt
    
artOutput = []
for name in ArtMap.keys():
    if 'False' in ArtMap[name]:
        ArtMap[name]['True']['haveFake'] = True
    else:
        ArtMap[name]['True']['haveFake'] = False
    artOutput.append(ArtMap[name]['True'])

artOutput = sorted(artOutput, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'realArtworkTitle', 'realArtworkTitle_tw', 'artist', 'museumDescription']
dictName += ['obtainedFrom', 'filename', 'haveFake', 'interact', 'itemCategory', 'themes', 'tag', 'size', 'realArtImage']

saveFile('art', artOutput, dictName)

get Furniture 1114
save art over


In [22]:
output = []

recipes = getItem('Recipes')

for item in recipes:

#     print(item['name'])
#     print('')
    pickData = {'type': 'recipes'}
    name = en2twFn(item['name'])

    pickData['id'] = int(item['content']['internalID'])
#     print(name['zh-tw'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    
    pickData['category'] = en2twFn(item['content']['itemCategory'], True)['zh-tw']
    pickData['filename'] = item['content']['itemImage'].split('/')[-1].replace('.png', '')
    pickData['DIY'] = True
    pickData['sell'] = item['content']['sell']
    pickData['materials'] = [{'count': item['count'], 'itemName': en2twFn(item['itemName'])['zh-tw']} for item in item['content']['materials']]
    pickData['obtainedFrom'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['obtainedFrom']]
#     pickData['sourceNotes'] = '' if item['content']['sourceNotes'] == '' else en2twFn(item['content']['sourceNotes'], True)['zh-tw']
    
    try:
        pickData['sourceNotes'] = '' if diyInfo['sourceNotes'] == '' else en2twFn(diyInfo['sourceNotes'], True)['zh-tw']
    except:
        pickData['sourceNotes'] = diyInfo['sourceNotes']
    
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])
dictName = ['id', 'name_e', 'name_j', 'name_c', 'category', 'filename', 'sell', 'materials', 'obtainedFrom', 'sourceNotes']

saveFile('recipes', output, dictName)

get Recipes 595
save recipes over


In [ ]:
output = []

villagers = getItem('Villagers')

for item in villagers:
    if item['category'] != "Villagers":
        continue

#     print(item['content'])
    pickData = {'type': 'villagers'}
    name = en2twFn(item['name'][0].upper() + item['name'][1:], 'onlySite')
#     print(item['name'], name['zh-tw'])

    pickData['id'] = int(item['id'])
    pickData['name_c'] = name['zh-tw']
    pickData['name_j'] = name['ja-jp']
    pickData['name_e'] = name['en-us']
    
    pickData['filename'] = item['content']['filename']
    pickData['birthday'] = item['content']['birthday']
    pickData['catchphrase'] = en2twFn(item['content']['catchphrase'])['zh-tw']
    pickData['colors'] = [en2twFn(obtainedFrom, True)['zh-tw'] for obtainedFrom in item['content']['colors']]
    pickData['favoriteSong'] = en2twFn(item['content']['favoriteSong'])['zh-tw']
    pickData['gender'] = en2twFn(item['content']['gender'], True)['zh-tw']
    
    pickData['hobby'] = en2twFn(item['content']['hobby'], True)['zh-tw']
    pickData['personality'] = en2twFn(item['content']['personality'], True)['zh-tw']

    pickData['species'] = en2twFn(item['content']['species'], True)['zh-tw']

    pickData['houseImage'] = item['content']['houseImage'].split('/')[-1].replace('.png', '')
    
    pickData['wallpaper'] = {
        'name': en2twFn(item['content']['wallpaper'])['zh-tw'], 
        'filename': name2fileNameMap[item['content']['wallpaper'][0].upper() + item['content']['wallpaper'][1:]]
    }
    pickData['flooring'] = {
        'name': en2twFn(item['content']['flooring'])['zh-tw'], 
        'filename': name2fileNameMap[item['content']['flooring'][0].upper() + item['content']['flooring'][1:]]
    }
    pickData['furnitureList'] = [idMap[int(furniture)] for furniture in item['content']['furnitureList']]
    
    output.append(pickData)

output = sorted(output, key=lambda k: k['id'])

dictName = ['id', 'name_e', 'name_j', 'name_c', 'filename', 'birthday', 'catchphrase', 'colors', 'favoriteSong']
dictName += ['gender', 'hobby', 'personality', 'species', 'houseImage', 'wallpaper', 'flooring', 'furnitureList']

saveFile('villagers', output, dictName)